**Develop a Chatbot for the E-commerce domain using the apparel dataset provided separately. It is expected that you are able to entertain multiple types of scenarios:~**

 - 1) A user starts a conversation by asking a few things in the beginning, like:
        User: Do you sell boys tops? (or similar question)
        Bot: Bot would say “yes” or “no”, depending upon the availability of the item? Also ask relevant attribute(s) to simplify the search and/or to engage the user (like which color or type you are looking for?)
        User: ......
        Bot: .......

- 2) The user directly provides a long sentence, such as:
        User: I need a t-shirt for my son in green or blue color
        Bot: respond with search results or ask about other attributes to simplify the search User: ........
        Bot: .......

- 3) The user may be very crude in his/her input and may only say “blue t-shirt for boys”. It is similar to category 2) above but has lesser greetings or formality from the user side. The bot may continue in its typical fashion, as was the case in 2).


The user may add more information in the subsequent responses as the discussion continues. Ensure that you maintain a memory to store the data received/extracted so far.


As discussed in the class, the final solution may use a hybrid approach that compromises both pattern-based and word2vec based search, which is perfectly fine.


_Deliverables: In addition to submitting a working code (preferably using Streamlit interface), briefly describe the approach you took._

# Product Chatbot Information Retrieval Recommendations Demo

In [1]:
from src.information_retrieval_model import Product_Information_Retrieval_Model

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/yawarkhalid/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yawarkhalid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model = Product_Information_Retrieval_Model()

/Users/yawarkhalid/miniforge3/envs/newenv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
import pandas as pd 
df = pd.read_excel("data/fashion_final.xlsx")
df.head()

/Users/yawarkhalid/miniforge3/envs/newenv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,SN,Gender,Category,SubCategory,ProductType,Colour,Usage,ProductTitle
0,1.0,Girls,Apparel,Topwear,Tops,White,Casual,Gini and Jony Girls Knit White Top
1,2.0,Girls,Apparel,Topwear,Tops,Black,Casual,Gini and Jony Girls Black Top
2,3.0,Girls,Apparel,Topwear,Tops,Blue,Casual,Gini and Jony Girls Pretty Blossom Blue Top
3,4.0,Girls,Apparel,Topwear,Tops,Pink,Casual,Doodle Kids Girls Pink I love Shopping Top
4,5.0,Girls,Apparel,Bottomwear,Capris,Black,Casual,Gini and Jony Girls Black Capris


In [4]:
request_body = "Pink tops for girls"
#recs = model.query_similar_products(message, 1)

i=0

recs = model.query_similar_products(request_body, 1)
product_name = recs.index.tolist()[i]
product_score  = round(recs.iloc[i].ensemble_similarity,4)

In [5]:
product_name

'hannah montana girl pink top'

In [6]:
product_score

0.8986

In [7]:
strings = ['do', 're', 'mi']
>>> ','.join(strings)

'do,re,mi'

In [8]:
product_name.replace(" ", "+")

'hannah+montana+girl+pink+top'

## Conversational Demo

In [9]:
import os
from src.information_retrieval_model import Product_Information_Retrieval_Model
import pandas as pd 
import emoji

import chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

from chatterbot.trainers import read_file

In [10]:
bot = ChatBot("Example Bot", read_only=False)
trainer = ListTrainer(bot)

#bot.storage.drop()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yawarkhalid/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yawarkhalid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yawarkhalid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
trainer.train([
'Hi',
'Hello',
'Sorry, we dont have this product',
'Yes, enter detailed description',
'Tell me more about it.',
'Sorry, we dont stock that item, add more description.',
'Im sorry, we dont have recommedations for that, you can try our webstore or add more description',
'Please enter more description',
'I hope ive helped.',
'Yes',
'No',
'You can exit any time',
'No Problem! Have a Good Day!'
])

List Trainer: [####################] 100%


In [12]:
model = Product_Information_Retrieval_Model()
df = pd.read_excel("data/fashion_final.xlsx")

name=input("Enter Your Name: ")

print("\nHi there! I'm a chatbot programmed to help you find your perfect fashion product! To get started, just respond with a description of what you are looking for. Be as detailed as you want!\n")

while True:
    request_body=input(name+':')
    
    if request_body=='Bye' or request_body =='bye':
        print('Bot: Thanks and enjoy your new purchase! If you want new recommendations, just reset me.')
        break
    
    else:
        recs = model.query_similar_products(request_body, 3)

        #format recommendation 1
        product_name = recs.index.tolist()[0]
        product_score1  = round(recs.iloc[0].ensemble_similarity,4)
        
        if product_score1 <=0.4:
            response=bot.get_response(request_body)
            print('Bot:',response)
            
        else:
        
            shopping_link = "https://www.google.com/search?q=" + product_name.replace(" ", "+")
            sms_message = emoji.emojize("Great! Here are your top recommendations :cherry_blossom:\n\n1) {} (match score={}). Shop for it here {}.\n ".format(product_name, product_score1, shopping_link))  


            #format recommendation 2
            product_name = recs.index.tolist()[1]
            product_score2  = round(recs.iloc[1].ensemble_similarity,4)
            shopping_link = "https://www.google.com/search?q=" + product_name.replace(" ", "+")
            sms_message = sms_message + emoji.emojize("\n2) You can also try :mushroom: {} (match score={}). Shop it here {}.\n".format(product_name, product_score2, shopping_link))


            #format recommendation 3
            product_name = recs.index.tolist()[2]
            product_score3  = round(recs.iloc[2].ensemble_similarity,4)
            shopping_link = "https://www.google.com/search?q=" + product_name.replace(" ", "+")
            sms_message = sms_message + emoji.emojize("\n3) Or try :bouquet: {} (match score={}. Shop it here {}. Enjoy!\n".format(product_name, product_score3 , shopping_link))
            sms_message = sms_message + emoji.emojize(":red_heart:")


            sms_message = sms_message + "\n\nThanks and enjoy your new purchase! If you want new recommendations, just type another product description.\nYou can type 'Bye' anytime to quit."
            response = sms_message
            print('\nBot:',response)

/Users/yawarkhalid/miniforge3/envs/newenv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/yawarkhalid/miniforge3/envs/newenv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Enter Your Name: Yawar

Hi there! I'm a chatbot programmed to help you find your perfect fashion product! To get started, just respond with a description of what you are looking for. Be as detailed as you want!

Yawar:white topwear for girls

Bot: Great! Here are your top recommendations 🌸

1) gini joni girl white top (match score=0.8002). Shop for it here https://www.google.com/search?q=gini+joni+girl+white+top.
 
2) You can also try 🍄 gini joni girl white top (match score=0.8002). Shop it here https://www.google.com/search?q=gini+joni+girl+white+top.

3) Or try 💐 gini joni girl white top (match score=0.8002. Shop it here https://www.google.com/search?q=gini+joni+girl+white+top. Enjoy!
❤️

Thanks and enjoy your new purchase! If you want new recommendations, just type another product description.
You can type 'Bye' anytime to quit.
Yawar:blue boys

Bot: Great! Here are your top recommendations 🌸

1) batman boy blue print t shirt (match score=0.8051). Shop for it here https://www.google